In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import wfdb
import ast
import os

In [13]:
# CSV dosyasını yükle
df = pd.read_csv('d:/ecg/data/raw/ptbxl/ptbxl_database.csv')

# İlk 1000 kaydı al
df_1000 = df.iloc[:1000]

def load_signal(record_name, data_path='d:/ecg/data/raw/ptbxl/'):
    full_path = os.path.join(data_path, record_name)
    record = wfdb.rdrecord(full_path)
    return record.p_signal

signals = []
labels = []
missing = 0
for idx, row in df_1000.iterrows():
    try:
        sig = load_signal(row['filename_hr'])
        signals.append(sig)
        labels.append(list(ast.literal_eval(row['scp_codes']).keys()))
    except FileNotFoundError:
        missing += 1
        print(f"Dosya bulunamadı: {row['filename_hr']}")
print(f"Toplam {missing} kayıt atlandı.")
signals = np.array(signals)

Dosya bulunamadı: records500/00000/00137_hr
Dosya bulunamadı: records500/00000/00139_hr
Dosya bulunamadı: records500/00000/00140_hr
Dosya bulunamadı: records500/00000/00141_hr
Dosya bulunamadı: records500/00000/00142_hr
Dosya bulunamadı: records500/00000/00143_hr
Dosya bulunamadı: records500/00000/00145_hr
Dosya bulunamadı: records500/00000/00456_hr
Dosya bulunamadı: records500/00000/00458_hr
Dosya bulunamadı: records500/00000/00459_hr
Dosya bulunamadı: records500/00000/00461_hr
Dosya bulunamadı: records500/00000/00462_hr
Toplam 12 kayıt atlandı.


In [14]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels)
print("Tüm hastalık türleri:", mlb.classes_)
print("signals shape:", signals.shape)
print("y shape:", y.shape)

Tüm hastalık türleri: ['1AVB' '3AVB' 'ABQRS' 'AFIB' 'AFLT' 'ALMI' 'AMI' 'ANEUR' 'ASMI' 'BIGU'
 'CLBBB' 'CRBBB' 'DIG' 'EL' 'HVOLT' 'ILBBB' 'ILMI' 'IMI' 'INJAL' 'INJAS'
 'INJIL' 'INJLA' 'INVT' 'IPLMI' 'IPMI' 'IRBBB' 'ISCAL' 'ISCAN' 'ISCAS'
 'ISCIL' 'ISCIN' 'ISCLA' 'ISC_' 'IVCD' 'LAFB' 'LAO/LAE' 'LMI' 'LNGQT'
 'LOWT' 'LPFB' 'LPR' 'LVH' 'LVOLT' 'NDT' 'NORM' 'NST_' 'NT_' 'PAC' 'PACE'
 'PVC' 'QWAVE' 'RAO/RAE' 'RVH' 'SARRH' 'SBRAD' 'SEHYP' 'SR' 'STACH' 'STD_'
 'SVTAC' 'TAB_' 'TRIGU' 'VCLVH']
signals shape: (988, 5000, 12)
y shape: (988, 63)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    signals, y, test_size=0.2, random_state=42
)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (790, 5000, 12)
y_train shape: (790, 63)


In [16]:
X_train_t = torch.tensor(X_train, dtype=torch.float32).permute(0,2,1)  # (batch, channels, seq_len)
X_test_t = torch.tensor(X_test, dtype=torch.float32).permute(0,2,1)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds = TensorDataset(X_test_t, y_test_t)

train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=8)

In [17]:
class InceptionBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.branch1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, padding=0)
        self.branch2 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.branch3 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.branch4 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        self.branch4_conv = nn.Conv1d(in_channels, out_channels, kernel_size=1, padding=0)
        self.bn = nn.BatchNorm1d(out_channels * 4)
        self.relu = nn.ReLU()
    def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4_conv(self.branch4(x))
        out = torch.cat([b1, b2, b3, b4], dim=1)
        out = self.bn(out)
        return self.relu(out)

class InceptionTime1D(nn.Module):
    def __init__(self, in_channels, n_classes):
        super().__init__()
        self.incept1 = InceptionBlock1D(in_channels, 16)
        self.incept2 = InceptionBlock1D(16*4, 32)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(32*4, n_classes)
    def forward(self, x):
        x = self.incept1(x)
        x = self.incept2(x)
        x = self.global_pool(x).squeeze(-1)
        return torch.sigmoid(self.fc(x))  # Çoklu etiket için sigmoid

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = InceptionTime1D(in_channels=12, n_classes=y.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")

Epoch 1/10, Loss: 0.0803
Epoch 2/10, Loss: 0.1027
Epoch 3/10, Loss: 0.2167
Epoch 4/10, Loss: 0.0318
Epoch 5/10, Loss: 0.0354
Epoch 6/10, Loss: 0.1228
Epoch 7/10, Loss: 0.1240
Epoch 8/10, Loss: 0.1626
Epoch 9/10, Loss: 0.0681
Epoch 10/10, Loss: 0.0719


In [19]:
model.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for xb, yb in test_dl:
        xb = xb.to(device)
        preds = model(xb).cpu().numpy()
        all_preds.append(preds)
        all_targets.append(yb.numpy())
all_preds = np.vstack(all_preds)
all_targets = np.vstack(all_targets)

# Her hastalık için threshold 0.5 ile tahmin
y_pred_bin = (all_preds > 0.5).astype(int)
print(classification_report(all_targets, y_pred_bin, target_names=mlb.classes_))

              precision    recall  f1-score   support

        1AVB       0.00      0.00      0.00         8
        3AVB       0.00      0.00      0.00         0
       ABQRS       0.00      0.00      0.00         3
        AFIB       0.00      0.00      0.00        14
        AFLT       0.00      0.00      0.00         1
        ALMI       0.00      0.00      0.00         1
         AMI       0.00      0.00      0.00         3
       ANEUR       0.00      0.00      0.00         0
        ASMI       0.00      0.00      0.00        20
        BIGU       0.00      0.00      0.00         0
       CLBBB       1.00      0.67      0.80         3
       CRBBB       0.00      0.00      0.00         5
         DIG       0.00      0.00      0.00         5
          EL       0.00      0.00      0.00         3
       HVOLT       0.00      0.00      0.00         2
       ILBBB       0.00      0.00      0.00         1
        ILMI       0.00      0.00      0.00         3
         IMI       0.00    

c:\Users\pasa-\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pasa-\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pasa-\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mod

In [20]:
for idx, disease in enumerate(mlb.classes_):
    tp = np.sum((all_targets[:,idx]==1) & (y_pred_bin[:,idx]==1))
    fn = np.sum((all_targets[:,idx]==1) & (y_pred_bin[:,idx]==0))
    fp = np.sum((all_targets[:,idx]==0) & (y_pred_bin[:,idx]==1))
    print(f"{disease}: TP={tp}, FN={fn}, FP={fp}")


1AVB: TP=0, FN=8, FP=0
3AVB: TP=0, FN=0, FP=0
ABQRS: TP=0, FN=3, FP=0
AFIB: TP=0, FN=14, FP=0
AFLT: TP=0, FN=1, FP=0
ALMI: TP=0, FN=1, FP=0
AMI: TP=0, FN=3, FP=0
ANEUR: TP=0, FN=0, FP=0
ASMI: TP=0, FN=20, FP=0
BIGU: TP=0, FN=0, FP=0
CLBBB: TP=2, FN=1, FP=0
CRBBB: TP=0, FN=5, FP=0
DIG: TP=0, FN=5, FP=0
EL: TP=0, FN=3, FP=0
HVOLT: TP=0, FN=2, FP=0
ILBBB: TP=0, FN=1, FP=0
ILMI: TP=0, FN=3, FP=0
IMI: TP=0, FN=16, FP=0
INJAL: TP=0, FN=0, FP=0
INJAS: TP=0, FN=1, FP=0
INJIL: TP=0, FN=0, FP=0
INJLA: TP=0, FN=0, FP=0
INVT: TP=0, FN=7, FP=0
IPLMI: TP=0, FN=0, FP=0
IPMI: TP=0, FN=1, FP=0
IRBBB: TP=0, FN=9, FP=0
ISCAL: TP=0, FN=1, FP=0
ISCAN: TP=0, FN=1, FP=0
ISCAS: TP=0, FN=0, FP=0
ISCIL: TP=0, FN=0, FP=0
ISCIN: TP=0, FN=3, FP=0
ISCLA: TP=0, FN=0, FP=0
ISC_: TP=1, FN=14, FP=1
IVCD: TP=0, FN=4, FP=0
LAFB: TP=0, FN=14, FP=0
LAO/LAE: TP=0, FN=2, FP=0
LMI: TP=0, FN=1, FP=0
LNGQT: TP=0, FN=2, FP=0
LOWT: TP=0, FN=3, FP=0
LPFB: TP=0, FN=4, FP=0
LPR: TP=0, FN=4, FP=0
LVH: TP=1, FN=17, FP=1
LVOLT: TP=0, F